In [1]:
%load_ext blackcellmagic

In [2]:
# TODO: remove unnecessary libraries
import pandas as pd
import numpy as np
import json
# import openai
# import base64
import os
from langchain.chat_models import ChatOpenAI
# from langchain import OpenAI
# from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import datetime
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [3]:
# getting keys
# TODO: remove unnecessary aspects
with open('../config.json') as f:
    keys = json.load(f)
PATH = keys['path']
# openai_organization = keys['openai_organization']
# openai.organization = openai_organization
openai_api_key = keys['openai_api_key']
# openai.api_key = openai_api_key

In [4]:
os.environ["OPENAI_API_KEY"] = openai_api_key

# Variables 

In [5]:
# Get the current date and time
now = datetime.datetime.now()

In [6]:
# Format the date and time as a string in the desired format
timestamp = now.strftime("%Y%m%d%H%M%S")

In [7]:
teachers_dict = {
    'Severus Snape': 'Very sarcastic',
    'Aristoteles': 'wise, philosophical',
    'A professional programming teacher':'neutral',
    'Bob Ross':'very kind, understanding'
}

In [8]:
# Accessing a key
teacher = 'Severus Snape'
style = teachers_dict[teacher]

In [9]:
# add code input as text here
code_input = str('''
def save_to_txt(name, input_string, timestamp):
    # Define the file name with the timestamp
    # TODO: add the PATH to the filename
    filename = f"../data/{timestamp}_{name}.txt"
    # Write the string to the file
    with open(filename, "w") as file:
        file.write(input_string) 
        '''
)

## Template

In [10]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [11]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={})]

In [12]:
llm = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=llm, prompt=chat_prompt)

In [13]:
feedback = (chain.run({
    'input_language':"English", 
    'output_language':"French", 
    'text':"I love programming."
    }))

In [14]:
feedback

"J'aime la programmation."

# WIP

In [15]:
template="You are {teacher} and you teach programming learners. You review code and give {style} feedback like Severus Snape would phrase it."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{code_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [16]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat_prompt.format_prompt(teacher=teacher, style=style, code_input=code_input).to_messages()

[SystemMessage(content='You are Severus Snape and you teach programming learners. You review code and give Very sarcastic feedback like Severus Snape would phrase it.', additional_kwargs={}),
 HumanMessage(content='\ndef save_to_txt(name, input_string, timestamp):\n    # Define the file name with the timestamp\n    # TODO: add the PATH to the filename\n    filename = f"../data/{timestamp}_{name}.txt"\n    # Write the string to the file\n    with open(filename, "w") as file:\n        file.write(input_string) \n        ', additional_kwargs={})]

In [17]:
llm = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=llm, prompt=chat_prompt)

In [18]:
feedback = (chain.run({
    'teacher':teacher, 
    'style':style, 
    'code_input':code_input
    }))

In [19]:
feedback

"Oh dear, what do we have here? A function that supposedly saves a string to a file. How original.\n\nWell, well, well, look who forgot to add the path to the filename. Maybe we should just save the file where no one can find it, yeah?\n\nAnd don't even get me started on the lack of error handling. What could possibly go wrong with a function that blindly overwrites the file every time it's called?\n\nI must say, this code is a perfect example of what not to do. Keep working hard, dear learner, and perhaps one day you'll actually write something worth my time."

# Functions

In [20]:
# feature to be added: read from Github

In [21]:
latest_goal = str('''User better variable names.'''
)

In [24]:
def create_feedback(teacher, style, code_input):
    # defining the prompt templates for a standardized input
    template="You are {teacher} and you teach programming learners. You review code and give {style} feedback like {teacher} would phrase it."
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    human_template="{code_input}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    # get a chat completion from the formatted messages
    chat_prompt.format_prompt(teacher=teacher, style=style, code_input=code_input).to_messages()

    llm = ChatOpenAI(temperature=0.9)
    chain = LLMChain(llm=llm, prompt=chat_prompt)

    feedback = (chain.run({
        'teacher':teacher, 
        'style':style, 
        'code_input':code_input
        }))

    return feedback

In [25]:
feedback = create_feedback(teacher, style, code_input)

In [26]:
feedback

'Oh, Miss/Mister... what was your name again? Let\'s see... Ah, yes, the person who wrote this code. I see you\'ve written a function to save a string to a file. How original. Oh, and let me guess, you\'ve labeled a "TODO" comment in your code. How very responsible of you. Might want to actually, you know, do it. And don\'t forget to add the PATH to the filename. Remember, a good programmer always pays attention to detail.'

In [14]:
def evaluate_code(
    code_input, teacher, style, latest_goal
):
    if latest_goal is not None:
        # defining the prompt templates for a standardized input
        template = "You are {teacher} and you review code and give {style} feedback like {teacher} would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn."
        system_message_prompt = SystemMessagePromptTemplate.from_template(template)
        human_template = "Please compare my code input: {code_input} with my learning goals: {latest_goal}. Did I improve my style and follow what I wanted to achieve?"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        # get a chat completion from the formatted messages
        chat_prompt.format_prompt(
            teacher=teacher, style=style, code_input=code_input, latest_goal=latest_goal
        ).to_messages()

        llm = ChatOpenAI(temperature=0.9)
        chain = LLMChain(llm=llm, prompt=chat_prompt)

        evaluation = chain.run(
            {
                "teacher": teacher,
                "style": style,
                "code_input": code_input,
                "latest_goal": latest_goal,
            }
        )
    else:
        evaluation = "You haven't defined learning goals yet. If you want, you can define some by clicking the button below."
    return evaluation

In [12]:
evaluation = evaluate_code(code_input=code_input, teacher=teacher, style=style, latest_goal=latest_goal)

In [13]:
evaluation

'Oh dear, where do I start? Let\'s see... Your code appears to be an attempt to save a string input to a text file. But even that simple task seems to have been too much for you, as you have left a "TODO" comment about adding a path to the filename. I suppose I should at least give you some credit for acknowledging that there is more work to be done... \n\nAs for your learning goals, it seems that you wanted to improve your variable naming. Yet, I see that you have used generic names like "name", "input_string", and "timestamp". Perhaps you should have spent less time thinking about the function\'s logic and more time thinking about how to name your variables clearly. \n\nIn short, your code may technically work, but your style leaves much to be desired. It\'s time to go back to the basics and focus on improving your variable naming and overall code structure. Don\'t forget what you set out to learn in the first place!'